In [4]:
import os
import torch
import torch.nn as nn
from torchvision import transforms

In [5]:
import sys
sys.path.insert(1, "../../")

In [6]:
from data_preprocessing import get_means, get_stds, ImageNetSubset
from Models.darknet19 import Darknet19
from evaluation import count_parameters

In [7]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu') 

In [8]:
data_path = '../../../dummy_datasets/'
norms_path = os.path.join(data_path, 'norms.json')

In [9]:
means = get_means(path=norms_path, train_loader=None)
stds = get_stds(path=norms_path, train_loader=None)

Means are: [0.4405549168586731, 0.4407285749912262, 0.4381718039512634]
stds are: [0.25142669677734375, 0.25270089507102966, 0.25131651759147644]


# Inference at 640x640

In [10]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((640, 640)),
                                      transforms.Normalize(mean=means, std=stds)])

In [11]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)

In [12]:
model = Darknet19(num_classes=1000, device=device, dtype=torch.float32)

In [13]:
img = train_dataset[0][0].unsqueeze(0)

In [14]:
from torch.profiler import profile, ProfilerActivity

In [15]:
with profile(activities=[ProfilerActivity.CPU],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=10, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../log/darknet19/inference_at_640'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=True) as prof:
    for i in range(12):
        out = model(img)
        prof.step()
print(prof.key_averages().table(row_limit=-1))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Total GFLOPs  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                    ProfilerStep*         0.43%      20.764ms       100.00%        4.779s     477.914ms     494.59 Mb      -4.35 Gb            10            --  
                       Conv block         1.87%      89.259ms        81.75%        3.907s      21.704ms       4.12 Gb           0 b           180            --  
                     aten::conv2d         0.05%       2.483ms        61.81%        2.954s      15.548ms       1.39 Gb           0 b           190       455.672  
                aten::convol

# Inference at 224x224

In [16]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((224, 224)),
                                      transforms.Normalize(mean=means, std=stds)])

In [17]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)

In [18]:
model = Darknet19(num_classes=1000, device=device, dtype=torch.float32)

In [19]:
img = train_dataset[0][0].unsqueeze(0)

In [20]:
from torch.profiler import profile, ProfilerActivity

In [21]:
with profile(activities=[ProfilerActivity.CPU],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=10, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../log/darknet19/inference_at_224'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=True) as prof:
    for i in range(12):
        out = model(img)
        prof.step()
print(prof.key_averages().table(row_limit=-1))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Total GFLOPs  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                    ProfilerStep*         1.64%      11.799ms       100.00%     718.170ms      71.817ms      53.61 Mb    -552.78 Mb            10            --  
                       Conv block         5.79%      41.586ms        82.39%     591.718ms       3.287ms     517.35 Mb           0 b           180            --  
                     aten::conv2d         0.19%       1.394ms        64.84%     465.640ms       2.451ms     174.13 Mb           0 b           190        55.820  
                aten::convol

# Inference at 224x224 using Ultralytics functions

In [22]:
from ultralytics_flops import get_flops, get_flops_with_torch_profiler

In [23]:
get_flops(model, imgsz=224)

5.618206368

In [24]:
get_flops_with_torch_profiler(model, imgsz=224)

5.581979648

# Count Parameters

In [207]:
count_parameters(model)

20841376